# Introduction

In this notebook, I'm gonna use chatGPT to generate prompts similar to the sample prompts we got from competition data. Since they did not disclose how the prompts in the test data were generated, it is safe to assume one way to generate those prompts is using another AI model like chatGPT. Although I should admit, after looking at the generated prompts, they are not as diversified or creative as human generated prompts. Maybe someone with better prompt engineering skills can generate better ones.

In [1]:
!pip install -U openai -q

In [2]:
import openai
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_key")
openai.organization = user_secrets.get_secret("openai_organization")

In [3]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path

sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')

In [4]:
import re
import time
from tqdm import tqdm

gpt_prompt = '''generate 15 prompts for a text to image AI model. 
Use a variety of random scenes, and range from fairly simple to fairly complex with multiple objects and modifiers. 
Use rare and common words to describe the scenes and objects in your prompts.
    Here are some examples of such prompts: 
    "hyper realistic photo of very friendly and dystopian crater"
    "ramen carved out of fractal rose ebony, in the style of hudson river school"
    "ultrasaurus holding a black bean taco in the woods, near an identical cheneosaurus"
    "a thundering retro robot crane inks on parchment with a droopy french bulldog"
    "portrait painting of a shimmering greek hero, next to a loud frill-necked lizard"
    "an astronaut standing on a engaging white rose, in the midst of by ivory cherry blossoms"
    "Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclaims ""Donuts. It's what's for dinner!"""'''


prompts = []
while len(prompts) < 5_000:
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                    {"role": "user", "content": gpt_prompt}
                ]
            )
    except Exception as e:
        time.sleep(1)
        continue
    new_p = response['choices'][0]['message']['content'].replace('\n\n', '\n').strip().split('\n')
    prompts += [re.sub(r"\d+\.[\) ]+", "", text) for text in new_p]
#     time.sleep(1)

### sample generated prompts:

In [5]:
prompts[:5]

['A majestic wolf howling at the moon, under a starry sky.',
 'A surreal landscape of floating islands, with giant carnival tents and flying balloons.',
 'A quaint countryside scene with a red barn, a rolling wheat field, and a vintage pickup truck.',
 'A futuristic cityscape with neon lights, hover cars, and a robotic police officer.',
 'An underwater scene with a coral reef, a school of colorful fish, and a mermaid lounging on a rock.']

In [6]:
data = pd.DataFrame(prompts, columns=['prompt'])
data.to_csv('gpt_prompts.csv', index_label='prompt_id')

## Generating Embeddings

In [7]:
st_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')
prompt_embeddings = st_model.encode(prompts)
data = pd.concat((data, pd.DataFrame(prompt_embeddings, columns=[f'eID_{i}' for i in range(384)])), axis=1)
data.to_csv('gpt_prompts_embeddings.csv', index_label='prompt_id')

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [8]:
data.sample(5)

,prompt,eID_0,eID_1,eID_2,eID_3,eID_4,eID_5,eID_6,eID_7,eID_8,...,eID_374,eID_375,eID_376,eID_377,eID_378,eID_379,eID_380,eID_381,eID_382,eID_383
1517,"""A tranquil scene of a still lake in the early...",-0.028207,0.084464,0.029139,0.041303,0.038876,0.040969,0.063307,0.009103,0.004186,...,-0.027252,0.045377,0.027175,0.033138,-0.087688,-0.034599,-0.008837,0.094204,-0.015810,-0.015743
4691,A futuristic cityscape of sleek glass skyscrap...,0.077169,0.036943,0.003312,0.000872,0.004201,-0.006136,0.028437,0.000572,-0.071288,...,-0.016884,-0.052017,0.010405,-0.022194,-0.059694,-0.031672,0.065708,0.031305,-0.047030,0.106347
3390,"A surrealist image of a giant flying octopus, ...",0.048939,0.063903,-0.004629,0.020849,0.019647,-0.032174,0.068103,-0.004191,0.008314,...,-0.010416,-0.063954,0.009055,-0.000013,-0.113333,0.043445,0.081409,-0.014877,0.029561,0.080582
445,"""A futuristic laboratory filled with glowing t...",-0.058321,0.070457,-0.011461,0.001655,-0.000174,-0.051805,0.053020,0.041047,-0.044619,...,0.003014,0.030116,0.012421,-0.017861,-0.161303,0.014283,0.108294,0.079497,0.014728,-0.095140
3059,"""A hauntingly beautiful sunset over a barren w...",0.092896,0.090350,0.049756,0.034834,0.105679,0.008816,0.071236,-0.055492,0.056846,...,-0.010621,-0.067462,0.043254,0.026749,-0.018511,-0.009758,0.124310,-0.004087,-0.001422,-0.059023
